## Imports

In [5]:
import numpy as np
import pandas as pd

## Get the raw data

In [24]:
file_path = './luz_pulsada_dados.csv'
def processed_data(file_path):
    data = pd.read_csv(file_path, delimiter='\t')
    data['pos'] = data['pos'] - data['pos'].iloc[0]
    data['freq'] = data['freq']/2
    return data

data = processed_data(file_path)

## Get the mean frequency for each bin of position

In [25]:
mean_freq = data.groupby('pos')['freq'].mean()

In [26]:
mean_freq

pos
-0.45    545.000000
-0.40    499.000000
-0.35    442.166667
-0.30    399.250000
-0.25    350.125000
-0.20    297.875000
-0.15    236.500000
-0.10    176.875000
-0.05    125.333333
 0.00     66.928571
Name: freq, dtype: float64